## Título
Las Multas del RGPD: ¿Una Herramienta Disuasoria Efectiva?

## Hipotesis
1) ¿Las multas reducen la reincidencia en infracciones del RGPD?

- Cuáles empresas reinciden
- Cual es la infracción mas repetida
- Distribución por países (suma de monto + cantidad de infracciones)
- Distribución por año (podría hacer un historico en flourish?)
- Como puedo clasificar las infracciones?

## Importación de los datos

In [1]:
import mysql.connector
import pandas as pd
import matplotlib.ticker as mticker
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Importo la base de datos 

bbdd = pd.read_excel("C:/Users/pacch/Desktop/Data/Análisis de Datos - IT Academy/proyecto/Docs a enviar/GDPR_fines.xlsx")

In [3]:
#Ahora importo una tabla creada por mí con los sectores de las empresas

sectores = pd.read_excel("C:/Users/pacch/Desktop/Data/Análisis de Datos - IT Academy/proyecto/Docs a enviar/Sectores.xlsx")

In [4]:
#Unimos las dos tablas

bbdd = pd.DataFrame(pd.merge(bbdd, sectores, on= 'ETid', how='left'))

In [5]:
#Pasaré a mayúscula la columna de 'ControllerProcessor' para unificar los nombres que tengan variantes con las mayúsculas y minúsculas
bbdd['ControllerProcessor'] = bbdd['ControllerProcessor'].str.upper()

## Exploración la base de datos

In [6]:
bbdd.head()

,ETid,Country,Date_of_Decision,Fine_€,ControllerProcessor,Quoted_Art,Type,Sector
0,ETid-1233,BELGIUM,2022-06-16,NaN,SA ROSSEL & CIE,Art 6 (1) a) GDPR Art 7 (1) GDPR Art 12 (1) GD...,Insufficient legal basis for data processing,Comunicación
1,ETid-1005,ITALY,2021-12-16,NaN,ENEL ENERGIA,Art 5 (1) a) d) GDPR Art 5 (2) GDPR Art 6 (1) ...,Insufficient legal basis for data processing,Energía
2,ETid-575,GERMANY,2021-03-03,NaN,PRIVATE INDIVIDUAL,Art 5 GDPR Art 32 GDPR,Non-compliance with general data processing pr...,Particular
3,ETid-224,POLAND,2020-03-04,NaN,SCHOOL IN GDANSK (DANZIG) (FINE IMPOSED AGAINS...,Art 5 GDPR Art 9 GDPR,Insufficient legal basis for data processing,Gobierno
4,ETid-126,ROMANIA,2019-11-28,NaN,ING BANK NV,Art 32 GDPR,Insufficient technical and organisational meas...,Finanzas


In [7]:
bbdd.shape

(2483, 8)

In [8]:
bbdd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2483 entries, 0 to 2482
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ETid                 2483 non-null   object 
 1   Country              2483 non-null   object 
 2   Date_of_Decision     2461 non-null   object 
 3   Fine_€               2379 non-null   float64
 4   ControllerProcessor  2358 non-null   object 
 5   Quoted_Art           2405 non-null   object 
 6   Type                 2459 non-null   object 
 7   Sector               2358 non-null   object 
dtypes: float64(1), object(7)
memory usage: 155.3+ KB


In [9]:
bbdd.isnull().sum()

ETid                     0
Country                  0
Date_of_Decision        22
Fine_€                 104
ControllerProcessor    125
Quoted_Art              78
Type                    24
Sector                 125
dtype: int64

## Ajuste de fecha en la columna Date_of_Decision

In [10]:
#Extraemos solo el año ya que en algunas columnas aparece mes-año, en otras solo año y en otras la fecha completa

bbdd['Date_of_Decision'] =  bbdd['Date_of_Decision'].apply(lambda x: str(x)[:4])

In [11]:
valores_unicos = bbdd['Date_of_Decision'].unique()
print(valores_unicos)

['2022' '2021' '2020' '2019' 'nan' '2023' '2024' '2018']


## Ajuste de valores null

In [12]:
bbdd['ControllerProcessor'] =bbdd['ControllerProcessor'].fillna('Desconocido')
bbdd['Sector'] =bbdd['Sector'].fillna('Desconocido')
bbdd['Type'] =bbdd['Type'].fillna('Desconocido')
bbdd['Date_of_Decision'] = bbdd['Date_of_Decision'].replace('nan', 'Desconocido')
bbdd.isnull().sum()

ETid                     0
Country                  0
Date_of_Decision         0
Fine_€                 104
ControllerProcessor      0
Quoted_Art              78
Type                     0
Sector                   0
dtype: int64

In [13]:
# Traducimos

traducciones = {
    'Non-compliance with general data processing principles': 'Incumplimiento de principios generales de procesamiento de datos',
    'Insufficient technical and organisational measures to ensure information security': 'Medidas técnicas y organizativas insuficientes para garantizar la seguridad de la información',
    'Insufficient legal basis for data processing': 'Base legal insuficiente para el procesamiento de datos',
    'Insufficient fulfilment of data subjects rights': 'Cumplimiento insuficiente de los derechos de los interesados',
    'Insufficient cooperation with supervisory authority': 'Cooperación insuficiente con la autoridad supervisora',
    'Insufficient fulfilment of information obligations': 'Cumplimiento insuficiente de las obligaciones de información',
    'Insufficient fulfilment of data breach notification obligations': 'Cumplimiento insuficiente de las obligaciones de notificación de brechas de datos',
    'Desconocido': 'Desconocido',
    'Insufficient involvement of data protection officer': 'Participación insuficiente del delegado de protección de datos',
    'Insufficient data processing agreement': 'Acuerdo de procesamiento de datos insuficiente'
}

bbdd['Type'] = bbdd['Type'].replace(traducciones)

print(bbdd['Type'].unique())

['Base legal insuficiente para el procesamiento de datos'
 'Incumplimiento de principios generales de procesamiento de datos'
 'Medidas técnicas y organizativas insuficientes para garantizar la seguridad de la información'
 'Cumplimiento insuficiente de los derechos de los interesados'
 'Desconocido'
 'Cumplimiento insuficiente de las obligaciones de información'
 'Cooperación insuficiente con la autoridad supervisora'
 'Cumplimiento insuficiente de las obligaciones de notificación de brechas de datos'
 'Acuerdo de procesamiento de datos insuficiente'
 'Participación insuficiente del delegado de protección de datos']


In [14]:
## Asignamos una abreviación a cada categoría

tipos_incumplimientos = {
    'Incumplimiento de principios generales de procesamiento de datos': 'Principios Generales',
    'Medidas técnicas y organizativas insuficientes para garantizar la seguridad de la información': 'Medidas Insuficientes',
    'Base legal insuficiente para el procesamiento de datos': 'Base Legal Insuficiente',
    'Cumplimiento insuficiente de los derechos de los interesados': 'Derechos de los Interesados',
    'Cooperación insuficiente con la autoridad supervisora': 'Cooperación Insuficiente',
    'Cumplimiento insuficiente de las obligaciones de información': 'Información Insuficiente',
    'Cumplimiento insuficiente de las obligaciones de notificación de brechas de datos': 'Notificación de Brechas',
    'Desconocido': 'Desconocido',
    'Participación insuficiente del delegado de protección de datos': 'No Participación del DPO',
    'Acuerdo de procesamiento de datos insuficiente': 'Acuerdo de procesamiento Insuficiente'}

bbdd['tipos_incumplimientos'] = bbdd['Type'].map(tipos_incumplimientos).astype(str)
bbdd.head()

,ETid,Country,Date_of_Decision,Fine_€,ControllerProcessor,Quoted_Art,Type,Sector,tipos_incumplimientos
0,ETid-1233,BELGIUM,2022,NaN,SA ROSSEL & CIE,Art 6 (1) a) GDPR Art 7 (1) GDPR Art 12 (1) GD...,Base legal insuficiente para el procesamiento ...,Comunicación,Base Legal Insuficiente
1,ETid-1005,ITALY,2021,NaN,ENEL ENERGIA,Art 5 (1) a) d) GDPR Art 5 (2) GDPR Art 6 (1) ...,Base legal insuficiente para el procesamiento ...,Energía,Base Legal Insuficiente
2,ETid-575,GERMANY,2021,NaN,PRIVATE INDIVIDUAL,Art 5 GDPR Art 32 GDPR,Incumplimiento de principios generales de proc...,Particular,Principios Generales
3,ETid-224,POLAND,2020,NaN,SCHOOL IN GDANSK (DANZIG) (FINE IMPOSED AGAINS...,Art 5 GDPR Art 9 GDPR,Base legal insuficiente para el procesamiento ...,Gobierno,Base Legal Insuficiente
4,ETid-126,ROMANIA,2019,NaN,ING BANK NV,Art 32 GDPR,Medidas técnicas y organizativas insuficientes...,Finanzas,Medidas Insuficientes


## Eliminación de columnas con las que no trabajaré

In [15]:
del bbdd['Quoted_Art']
del bbdd['Type']
bbdd.head()

,ETid,Country,Date_of_Decision,Fine_€,ControllerProcessor,Sector,tipos_incumplimientos
0,ETid-1233,BELGIUM,2022,NaN,SA ROSSEL & CIE,Comunicación,Base Legal Insuficiente
1,ETid-1005,ITALY,2021,NaN,ENEL ENERGIA,Energía,Base Legal Insuficiente
2,ETid-575,GERMANY,2021,NaN,PRIVATE INDIVIDUAL,Particular,Principios Generales
3,ETid-224,POLAND,2020,NaN,SCHOOL IN GDANSK (DANZIG) (FINE IMPOSED AGAINS...,Gobierno,Base Legal Insuficiente
4,ETid-126,ROMANIA,2019,NaN,ING BANK NV,Finanzas,Medidas Insuficientes


## Traducción del nombre de las columnas

In [16]:
bbdd = bbdd.rename(columns={'ETid': 'Id', 'Country': 'País', 'Date_of_Decision' : 'Año_decision', 'Fine_€': 'Multa', 'ControllerProcessor': 'Responsable_tratamiento', 'tipos_incumplimientos': 'Tipo_incumplimiento'})
bbdd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2483 entries, 0 to 2482
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Id                       2483 non-null   object 
 1   País                     2483 non-null   object 
 2   Año_decision             2483 non-null   object 
 3   Multa                    2379 non-null   float64
 4   Responsable_tratamiento  2483 non-null   object 
 5   Sector                   2483 non-null   object 
 6   Tipo_incumplimiento      2483 non-null   object 
dtypes: float64(1), object(6)
memory usage: 135.9+ KB


In [17]:
bbdd.isnull().sum()

Id                           0
País                         0
Año_decision                 0
Multa                      104
Responsable_tratamiento      0
Sector                       0
Tipo_incumplimiento          0
dtype: int64

In [18]:
bbdd.sort_values(by='Multa', ascending=False).head(15)

,Id,País,Año_decision,Multa,Responsable_tratamiento,Sector,Tipo_incumplimiento
2385,ETid-1844,IRELAND,2023,1.200000e+09,FACEBOOK,Redes Sociales,Base Legal Insuficiente
2384,ETid-778,LUXEMBOURG,2021,7.460000e+08,AMAZON,Comercio,Principios Generales
2383,ETid-1373,IRELAND,2022,4.050000e+08,FACEBOOK,Redes Sociales,Principios Generales
2382,ETid-875,IRELAND,2021,3.900000e+08,FACEBOOK,Redes Sociales,Información Insuficiente
2381,ETid-1543,IRELAND,2023,3.900000e+08,FACEBOOK,Redes Sociales,Principios Generales
2380,ETid-2032,IRELAND,2023,3.450000e+08,TIKTOK,Redes Sociales,Principios Generales
2379,ETid-2469,IRELAND,2024,3.100000e+08,LINKEDIN,Redes Sociales,Base Legal Insuficiente
2378,ETid-2447,THE NETHERLANDS,2024,2.900000e+08,UBER,Transporte,Principios Generales
2377,ETid-1502,IRELAND,2022,2.650000e+08,FACEBOOK,Redes Sociales,Medidas Insuficientes
2376,ETid-820,IRELAND,2021,2.250000e+08,WHATSAPP IRELAND LTD,Mensajería Instantánea,Información Insuficiente


## Análisis de los datos

### ¿Cuantos responsables de tratamiento tenemos?

In [19]:
num_responsables = len(bbdd['Responsable_tratamiento'].unique())
print(f'Tenemos {num_responsables} responsables del tratamiento')

Tenemos 1522 responsables del tratamiento


### ¿Cuántos responsables reincidentes tenemos?

In [20]:
conteo_reincidentes= bbdd.groupby('Responsable_tratamiento').agg(count=('Id', 'nunique'), sum_Multa=('Multa', 'sum')).sort_values(by='count', ascending=False)
conteo_reincidentes = pd.DataFrame(conteo_reincidentes)
conteo_reincidentes['count'] = conteo_reincidentes['count'].astype(int)
conteo_reincidentes1 = conteo_reincidentes[conteo_reincidentes['count'] > 1]
pd.options.display.float_format = '{:.2f}'.format  #Para convertir la notación cientifica
num_reincidentes = len(conteo_reincidentes1)
print(f'Tenemos {num_reincidentes} reincidentes')


#Tenemos 137 reincidentes, tomando el cuenta que tenemos a private individual, desconocido, y nombres genericos de empresas

Tenemos 137 reincidentes


In [21]:
conteo_reincidentes1.describe()

,count,sum_Multa
count,137.00,137.00
mean,8.01,37071341.82
std,23.63,252881900.71
min,2.00,0.00
25%,2.00,12000.00
50%,3.00,43000.00
75%,4.00,738235.00
max,220.00,2818051000.00


### Como vemos que son muchos, filtramos por >3 y vemos que se reduce a 47

In [22]:
conteo_reincidentes3 = conteo_reincidentes[conteo_reincidentes['count'] > 3]
num_reincidentess3 = len(conteo_reincidentes3)
print(f'Tenemos {num_reincidentess3} reincidentes con más de tres multas')

# Tenemos 47 responsables con mas de 3 multas, es decir, más de 2 reincidencias

Tenemos 47 reincidentes con más de tres multas


In [23]:
conteo_reincidentes3.head()

,count,sum_Multa
Responsable_tratamiento,,
PRIVATE INDIVIDUAL,220,336410.00
Desconocido,125,1966236.00
VODAFONE,96,30946051.00
COMPANY,74,1661345.00
HOMEOWNER ASSOCIATION,36,65400.00


### Top 10 de encargados de tratamiento que más han pagado en multas

In [24]:
Suma_multas = bbdd[bbdd['Multa'].notna()] 
Suma_multas = Suma_multas.groupby('Responsable_tratamiento').agg(count=('Id', 'nunique'), sum_Multa=('Multa', 'sum')).sort_values(by='sum_Multa', ascending=False)
top_mas_sum = Suma_multas.head(10)
top_mas_sum = pd.DataFrame(top_mas_sum)
lista_nombres = top_mas_sum.index.tolist()
top_mas_sum 

,count,sum_Multa
Responsable_tratamiento,,
FACEBOOK,9,2818051000.00
AMAZON,5,780060000.00
TIKTOK,3,360250000.00
LINKEDIN,1,310000000.00
UBER,2,300000000.00
WHATSAPP IRELAND LTD,2,230500000.00
GOOGLE,7,215600028.00
ENEL ENERGIA,5,79123000.00
CLEARVIEW AL INC,4,69000000.00


- Facebook (Meta) -	Estados Unidos	- Multinacional (Big Tech)
- Amazon -	Estados Unidos	- Multinacional (Big Tech)
- TikTok (ByteDance) -	China	- Multinacional (Big Tech)
- LinkedIn -	Estados Unidos	- Multinacional (Propiedad de Microsoft)
- Uber	- Estados Unidos	- Multinacional (Gig Economy)
- WhatsApp -	Estados Unidos -	Multinacional (Big Tech)
- Google (Alphabet Inc.)	- Estados Unidos	- Multinacional (Big Tech)
- Enel Energia	- Italia	- Multinacional (Sector energético)
- Clearview AI Inc	- Estados Unidos	- Empresa de tecnología de reconocimiento facial
- Criteo	- Francia	- Multinacional (Publicidad digital)

## Top 10 de responsables que menos han pagado en multas

In [25]:
cinco_menos_sum = Suma_multas.sort_values(by='sum_Multa', ascending=False).tail(10)
cinco_menos_sum = pd.DataFrame(cinco_menos_sum)
cinco_menos_sum

,count,sum_Multa
Responsable_tratamiento,,
CBHNOS SL,1,300.00
REPRESENTATIVE OF A LOCAL GOVERNMENT,1,290.00
DIRECTORATE OF SOCIAL AND CHILD WELFARE INSTITUTIONS OF THE FERENCVAROS DISTRICT OF BUDAPEST,1,286.00
MH VILASECA SL,1,240.00
CARROZADOS TECAI SL,1,180.00
Y OTRO MAS CB,1,180.00
SUPER 24H LOS ROSALES SL,1,180.00
EURO DONER KEBAB,1,180.00
INMUR JOYEROS SL,1,180.00


Si filtro por ultimos 5 estos son los resultados:

- Y OTRO MÁS CB	(España): Microempresa restaurante y puestos de comida	
- SUPER 24H LOS ROSALES SL (España): Comercio minorista de alimentos, bebidas y tabaco
- EURO DONER KEBAB (España): Autónomo - Restaurante especializado en kebabs
- INMUR JOYEROS SL	(España)	Joyería, comercio minorista de joyas y relojes

Si filtro por ultimos 50 esto es lo que he encontrado sobre las entidades:

De las entidades mencionadas, Festina Lotus SA destaca como una empresa reconocida con presencia internacional. Fundada en 1980, es una compañía española dedicada a la fabricación y distribución de relojes y joyas, conocida por marcas como Festina, Lotus y Jaguar. Su sede central se encuentra en Barcelona, y ha logrado una notable presencia en mercados europeos y latinoamericanos.

Las demás entidades listadas parecen ser empresas o asociaciones de ámbito local o regional, sin un reconocimiento significativo a nivel nacional o internacional.

## Top 10 responsables reincidentes con más multas impuestas

In [26]:
count_multas = bbdd[bbdd['Multa'].notna()]
topmas_count = count_multas.groupby('Responsable_tratamiento').agg(count=('Id', 'nunique'), sum_Multa=('Multa', 'sum')).sort_values(by='count', ascending=False)
topmas_count = topmas_count[topmas_count['count'] > 1]
topmas_count = topmas_count.head(25)
topmas_count = pd.DataFrame(topmas_count)
topmas_count

# Hago un head de 25 para poder sacar al menos 10 empresas, ya que tenemos varias categorías de reponsables anonimizados

,count,sum_Multa
Responsable_tratamiento,,
PRIVATE INDIVIDUAL,206,336410.00
Desconocido,107,1966236.00
VODAFONE,96,30946051.00
COMPANY,61,1661345.00
HOMEOWNER ASSOCIATION,36,65400.00
POLICE,23,31678.00
XFERA MOVILES SA,17,898000.00
TELEFONICA,17,1808000.00
PHYSICIAN,15,45800.00


In [27]:
bbdd.to_excel('bbdd.xlsx', index=False)

## Top 10 responsables reincidentes con menos multas impuestas

Aquí tenemos una situación más complicada,  ya que tenemos 42 entidades empatadas con la misma cantidad al final de la tabla, es decir, con 2 multas. Sacaré una muestra de 5 entidades con la menor cantidad pagada en multas o otra muestra con los que teniendo 1 reincidenta, han pagado mas multas

In [28]:
#5 entidades con una reincidencia que menos han pagado
topmenos_count = count_multas.groupby('Responsable_tratamiento').agg(count=('Id', 'nunique'), sum_Multa=('Multa', 'sum'))
topmenos_count = topmenos_count[topmenos_count['count'] == 2]
topmenos_count5menos = topmenos_count.sort_values(by='sum_Multa', ascending=False)
topmenos_count5menos = pd.DataFrame(topmenos_count5menos)
topmenos_count5menos.tail(7)

,count,sum_Multa
Responsable_tratamiento,,
LISMARTSA SL,2,3000.00
PROPERTY OWNER,2,3000.00
EXPLOTACIONES HOSTELERAS Y DE OCIO ALBACETEÑAS SL,2,2500.00
NEIGHBORHOOD COMMUNITY,2,2500.00
ALBEN AIRPORT FACILITIES SL,2,2000.00
MEDICOVER SRL,2,2000.00
NN ASIGURĂRI DE VIAȚĂ SA,2,2000.00


In [29]:
#5 entidades con una reincidencia que más han pagado
topmenos_count5mas = topmenos_count.sort_values(by='sum_Multa', ascending=True)
topmenos_count5mas = pd.DataFrame(topmenos_count5mas)
topmenos_count5mas.tail()

,count,sum_Multa
Responsable_tratamiento,,
ENDESA ENERGÍA,2,6160000.00
ENI GAS E LUCE,2,11500000.00
CLEARVIEW AI INC,2,30510000.00
WHATSAPP IRELAND LTD,2,230500000.00
UBER,2,300000000.00


### Evolución de las multas de las empresas que más reincidencias tienen

In [30]:
lista_nombres = ['ORANGE SAU', 'TELEKOM', 'FACEBOOK', 'CAIXABANK SA', 'BBVA SA', 'DIGI TELECOM SL', 'WEBSITE OPERATOR', 'XFERA MOVILES SA', 'TELEFONICA', 'VODAFONE']
bbdd_filtrada = bbdd[bbdd['Responsable_tratamiento'].isin(lista_nombres)]
bbdd_filtrada = bbdd_filtrada.sort_values(by=['Año_decision','Responsable_tratamiento'], ascending=[True, True])
del bbdd_filtrada['Sector']
del bbdd_filtrada['País']

bbdd_filtrada.to_excel('evolucion_masmultas.xlsx', index=False)

PermissionError: [Errno 13] Permission denied: 'evolucion_masmultas.xlsx'

## Sectores con mas multas

In [35]:
# ¿Cuántos sectores tenemos?

num_sectores = len(bbdd['Sector'].unique())
print(f'Tenemos {num_sectores} sectores')

Tenemos 191 sectores


### Top 10 de sectores que más han tenido multas

In [36]:
multas_sector = bbdd.groupby('Sector')['Id'].count().sort_values(ascending=False).reset_index()
multas_sector_head = multas_sector.head(15)
multas_sector_head

,Sector,Id
0,Particular,217
1,Telecomunicaciones,192
2,Salud,152
3,Tecnología,132
4,Comercio,128
5,Desconocido,125
6,Gobierno,112
7,Finanzas,83
8,Empresa,81
9,Energía,73


In [37]:
rrss = multas_sector[multas_sector['Sector'] == 'Redes Sociales']
print(rrss)

            Sector  Id
34  Redes Sociales  15


## Media y medianas de multas por sector

In [38]:
bbdd.describe()

,Multa
count,2379.00
mean,2404634.97
std,35004423.13
min,28.00
25%,2000.00
50%,6700.00
75%,40000.00
max,1200000000.00


In [39]:
medias_sector = bbdd.copy()
medias_sector = medias_sector[medias_sector['Multa'].notna()]
medias_sector['Multa'] = medias_sector['Multa'].astype(float)
medias_sector = medias_sector.groupby('Sector')['Multa'].mean().sort_values(ascending=False).head(10)
medias_sector = medias_sector.reset_index()
medias_sector

,Sector,Multa
0,Redes Sociales,249196500.00
1,Mensajería Instantánea,77100000.00
2,Motores de Búsqueda,31950003.50
3,Transporte,7500418.21
4,Software,6980000.00
5,Comercio,6310301.59
6,Utilities,5750000.00
7,Aerolínea,4428800.00
8,Estadísticas,4300000.00
9,Hospitalidad,2560148.88


In [40]:
medianas_sector = bbdd.copy()
medianas_sector  = medianas_sector [medianas_sector ['Multa'].notna()]
medianas_sector ['Multa'] = medianas_sector ['Multa'].astype(float)
medianas_sector  = medianas_sector .groupby('Sector')['Multa'].median().sort_values(ascending=False).head(10)
medianas_sector  = medianas_sector .reset_index()
medianas_sector 

,Sector,Multa
0,Redes Sociales,178000000.00
1,Motores de Búsqueda,25000000.00
2,Software,6980000.00
3,Utilities,5750000.00
4,Mensajería Instantánea,5500000.00
5,Estadísticas,4300000.00
6,Biología,1500000.00
7,Comecio electrónico,1405000.00
8,Organización,487500.00
9,Servicios deportivos,455000.00


## Análisis por país

In [41]:
#Ordenados por suma de multas pagadas

conteo_país = bbdd[bbdd['Multa'].notna()]
conteo_país_monto= conteo_país.groupby('País')['Multa'].agg(['count', 'sum']).sort_values(by='sum', ascending=False)
conteo_país_monto

,count,sum
País,,
IRELAND,29,3646363400.00
LUXEMBOURG,32,746314000.00
FRANCE,54,371820200.00
THE NETHERLANDS,28,346030500.00
ITALY,388,237287258.00
SPAIN,899,82411890.00
UNITED KINGDOM,15,75541500.00
GERMANY,131,55905123.00
GREECE,70,34298540.00


In [42]:
#Ordenados por cantidad de multas pagadas

conteo_país_conteo= conteo_país.groupby('País')['Multa'].agg(['count', 'sum']).sort_values(by='count', ascending=False)
conteo_país_conteo

,count,sum
País,,
SPAIN,899,82411890.00
ITALY,388,237287258.00
ROMANIA,192,1196950.00
GERMANY,131,55905123.00
POLAND,81,5317479.00
GREECE,70,34298540.00
HUNGARY,68,2518861.00
FRANCE,54,371820200.00
NORWAY,54,12265850.00
